In [ ]:
# for creating a responsive plot
%matplotlib widget

# importing required libraries
import pypulse as pyp
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import numpy as np
import pandas as pd
import glob

import sys

# Configure Plotly to be rendered inline in the notebook.
plotly.offline.init_notebook_mode()

In [ ]:
file = "./results/pol_calibrated/L_band_meerguard_pazr/0.0_sigma/features.pkl"
data = pd.read_pickle(file)
filtered_data = data.loc[~((data['Width'] == 0.0))]

In [ ]:
filtered_data = filtered_data[filtered_data['Energy'] < 100.0]
filtered_data = filtered_data[filtered_data['Energy'] > -10.0]
#data.to_csv("./results/820_band/0.0_sigma/features.csv")

In [ ]:
fig = px.scatter_3d(filtered_data,
                    x='Width', y='Energy', z='Amp',
                    template="plotly")

name = 'L Band'
# Default parameters which are used when `layout.scene.camera` is not provided
camera = dict(
    eye=dict(x=1.5, y=2, z=1.5))
fig.update_traces(marker_size = 4)

fig.update_layout(scene_camera=camera, title=name)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(autosize=False, width=600, height=600)

fig.update_layout(
    font=dict(
#        family="Courier New, monospace",
        size=15,  # Set the font size here
#        color="RebeccaPurple"
    )
)

fig.write_image("plots/L_band_3d_features.pdf")
fig.show()

In [ ]:
# creating figure

trace = go.Scatter3d(clustered_data,
                     x='Width', y='Amp', z='Energy', mode='markers',
                     marker={'size': 10, 'opacity': 0.8,})

layout = go.Layout(margin={'l': 0, 'r': 0, 'b': 0, 't': 0})

plot_figure = go.Figure(data=[trace], layout=layout)

#fig.update_layout(legend_title_text = file[:-5])
plot_figure.update_layout(scene_aspectmode='auto')
plot_figure.update_layout(scene = dict(
#                    xaxis_title='Width',
#                    yaxis_title='Energy',
                    zaxis_title='Amp'),
                    width=700,
                    margin=dict(r=20, b=10, l=10, t=10))

# Render the plot.
plotly.offline.iplot(plot_figure)

## Plot Kmeans-clustered data

In [ ]:
clustered_data = pd.read_pickle("/Users/vs4793/PycharmProjects/highfluencetiming/results/pol_calibrated/L_band_meerguard_pazr/0.0_sigma/Kmeans/8_kmeans_clusters.pkl").sort_values('Cluster')
clustered_data = clustered_data[clustered_data['Energy'] < 100.0]
clustered_data = clustered_data[clustered_data['Energy'] > -10.0]

In [ ]:
cmap = [px.colors.qualitative.Plotly[i] for i in [3, 0, 5, 2, 9, 4, 1, 8]]

In [ ]:
# Post-clustering figure

#cmap = sns.color_palette('colorblind')

#labels = clustered_data[:, 4].astype(int)
#unique_labels = np.unique(labels)

fig = px.scatter_3d(clustered_data, x='Width', y='Energy', z='Amp',
              color='Cluster', template="plotly", color_discrete_sequence=cmap)

fig.update_layout(legend = dict(orientation="v", x = 0.7, y = 0.72, traceorder='normal'))
name = 'L Band'
# Default parameters which are used when `layout.scene.camera` is not provided
camera = dict(
    eye=dict(x=-1.5, y=-2.0, z=0.5))
fig.update_traces(marker_size = 4)

fig.update_layout(scene_camera=camera, title=name)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(autosize=False, width=600, height=600)

fig.update_layout(
    scene = dict(
                    xaxis_title='Width',
                    yaxis_title='Energy',
                    zaxis_title='Amplitude'),
    font=dict(
#        family="Courier New, monospace",
        size=15,  # Set the font size here
#        color="RebeccaPurple"
    )
)

fig.write_image("plots/L_band_3d_clustered.pdf")
fig.show()

## Plot OPTICS-clustered data

In [ ]:
clustered_data = pd.read_pickle("./results/pol_calibrated/820_band_meerguard_pazr/0.0_sigma/OPTICS_min_cluster_size_0.03/0.11_maxeps/0.11_OPTICS_features.pkl").sort_values('Cluster')

fig = px.scatter_3d(clustered_data, x='Width', y='Energy', z='Amp', opacity=0.3,
              color='Cluster', template="plotly", color_discrete_sequence=cmap)

fig.update_layout(legend = dict(orientation="v", x = 0.7, y = 0.72, traceorder='normal'))

# Default parameters which are used when `layout.scene.camera` is not provided
camera = dict(
    eye=dict(x=-1.5, y=-2.0, z=0.5))
fig.update_traces(marker_size = 4)

fig.update_layout(scene_camera=camera)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(autosize=False, width=600, height=600)

fig.update_layout(
    scene = dict(
                    xaxis_title='Width',
                    yaxis_title='Energy',
                    zaxis_title='Amplitude'),
    font=dict(
#        family="Courier New, monospace",
        size=15,  # Set the font size here
#        color="RebeccaPurple"
    )
)

#fig.write_image("plots/OPTICS_3d_clustered.pdf")
fig.show()

# Plot examples from each cluster

In [ ]:
clustered_data = pd.read_pickle("/Users/vs4793/PycharmProjects/highfluencetiming/results/pol_calibrated/820_band_meerguard_pazr/0.0_sigma/Kmeans/7_kmeans_clusters.pkl")
clustered_results = pd.read_pickle("/Users/vs4793/PycharmProjects/highfluencetiming/results/pol_calibrated/820_band_meerguard_pazr/0.0_sigma/Kmeans/7_clusters/7_clusters_results.plk")
unnormalized_data = pd.read_pickle("/Users/vs4793/PycharmProjects/highfluencetiming/results/pol_calibrated/820_band_meerguard_pazr/0.0_sigma/unnormalized_data.pkl")

In [ ]:
def clean_artifacts(cluster_average_pulse, clean_window, delta:int = 20):
    
    interpolating_bins = np.r_[clean_window[0]-delta : clean_window[0] , clean_window[1]+1: clean_window[1]+1 + delta]
    
    cluster_average_pulse[clean_window[0]:clean_window[1]+1] = np.interp(x=np.arange(clean_window[0], clean_window[1]+1, 1),
                                                                         xp=interpolating_bins, fp=cluster_average_pulse[interpolating_bins])

    return cluster_average_pulse

In [ ]:
n_clusters : int = len(clustered_data.Cluster.unique())
bins = np.arange(1, 513, 1, dtype=int)

fig, axs = plt.subplots(nrows=6, ncols=4, sharex=True, figsize=(16, 16),
                        gridspec_kw=dict(hspace=0.0, height_ratios=[1, 1, 1, 1, 1, 1.5]))

sns.set_style('ticks')
sns.set_context("paper", font_scale = 2.0)
#cmap = sns.color_palette('colorblind')
cmap =  sns.color_palette(palette='Paired')[0:4] + sns.color_palette(palette='Paired')[6:8] + sns.color_palette(palette='Paired')[4:6]

#fig.suptitle(f'K-means with $k={n_clusters}$ clusters', fontsize=32)

for column, cluster_index in enumerate([2, 3, 4, 6]):
    
     # Isolate the single pulses in the cluster
    cluster_sp_times = clustered_data[clustered_data['Cluster'] == str(cluster_index)].index.to_numpy()
    cluster_average_pulse = np.average(unnormalized_data.loc[cluster_sp_times].to_numpy(), axis=0)
#    cluster_avg_sp = pyp.SinglePulse(cluster_average_pulse, opw=np.arange(0, 100))
#    cluster_avg_sp.remove_baseline(save=True)
     
    for row, sp_t in enumerate(cluster_sp_times[1:7]):
        
        if row == 0:
            axs[row, column].set_title(f'Cluster {cluster_index} \n $w={round(clustered_results["1/sigma^2"].iloc[cluster_index], 2)}$', fontsize=28)

        if row == 5:
            smoothed_cluster = clean_artifacts(cluster_average_pulse, [224, 227])
            axs[row, column].plot(smoothed_cluster, lw=2.0, color=cmap[2 * column + 1])
            axs[row, column].set_xlabel("Phase Bins")
            
        else:
            axs[row, column].plot(unnormalized_data.loc[[sp_t]].transpose().to_numpy(), color=cmap[2 * column])
             
            if column == 0:
                axs[row, column].set_ylabel("Intensity")

plt.tight_layout()
plt.savefig("./plots/820_band_clusters_examples.pdf")
plt.show()

In [ ]:
clustered_data = pd.read_pickle("/Users/vs4793/PycharmProjects/highfluencetiming/results/pol_calibrated/L_band_everything/0.0_sigma/Kmeans/8_kmeans_clusters.pkl")
unnormalized_data = pd.read_pickle("/Users/vs4793/PycharmProjects/highfluencetiming/results/pol_calibrated/L_band_everything/0.0_sigma/unnormalized_data.pkl")

In [ ]:
n_clusters : int = len(clustered_data.Cluster.unique())
bins = np.arange(1, 513, 1, dtype=int)

fig, axs = plt.subplots(nrows=6, ncols=8, sharex=True, figsize=(32, 16),
                        gridspec_kw=dict(hspace=0.0, height_ratios=[1, 1, 1, 1, 1, 1.5]))

sns.set_style('ticks')
sns.set_context("paper", font_scale = 2.0)
cmap = sns.color_palette()
#cmap =  sns.color_palette(palette='Paired')[0:4] + sns.color_palette(palette='Paired')[6:8] + sns.color_palette(palette='Paired')[4:6]

#fig.suptitle(f'K-means with $k={n_clusters}$ clusters', fontsize=32)

for column, cluster_index in enumerate([0, 1, 2, 3, 4, 5, 6, 7]):
    
     # Isolate the single pulses in the cluster
    cluster_sp_times = clustered_data[clustered_data['Cluster'] == str(cluster_index)].index.to_numpy()
    cluster_average_pulse = np.average(unnormalized_data.loc[cluster_sp_times].to_numpy(), axis=0)
#    cluster_avg_sp = pyp.SinglePulse(cluster_average_pulse, opw=np.arange(0, 100))
#    cluster_avg_sp.remove_baseline(save=True)
     
    for row, sp_t in enumerate(cluster_sp_times[1:7]):
        
        if row == 0:
            axs[row, column].set_title(f'Cluster {cluster_index} \n $n={len(cluster_sp_times)}$', fontsize=28)

        if row == 5:
            smoothed_cluster = clean_artifacts(cluster_average_pulse, [224, 238])
#            smoothed_cluster = clean_artifacts(cluster_average_pulse, [224, 227])
            axs[row, column].plot(smoothed_cluster, lw=2.0) #, color=cmap[2 * column + 1])
            axs[row, column].set_xlabel("Phase Bins")
            
        else:
            axs[row, column].plot(unnormalized_data.loc[[sp_t]].transpose().to_numpy()) #, color=cmap[2 * column])
             
            if column == 0:
                axs[row, column].set_ylabel("Intensity")

plt.tight_layout()
#plt.savefig("./plots/L_band_clusters_examples.pdf")
plt.show()

In [ ]:
ds = pd.read_pickle("./results/820_band/dynamic_spectrum.pkl")
display(ds)

fig = px.imshow(ds, labels=dict(x="Time (s)", y="Frequency channel (MHz)"), aspect="auto",
                color_continuous_scale=px.colors.sequential.Inferno)
fig.show()

In [ ]:
file = "./data/820_band/GUPPI_57785_J2145-0750_0004_0001.ar"
channels = pyp.Archive(file, prepare=False, center_pulse=False, baseline_removal=False,
                         lowmem=True, verbose=False).getAxis(flag="F", edges=True)
print(len(channels))
print(len(channels[64:-32]))